In [1]:
import os
import logging, timeit
from btEngine2.DataLoader import DataLoader
from btEngine2.MarketData import MarketData
from btEngine2.TradingRule import TradingRule



# Define paths and configurations for DataLoader
ticker_csv_path = r'G:\Projects\BackTesting1.0\Data\Inputs\TickerList-Futs.csv'
save_directory = r"G:\Projects\BackTesting1.0\Data\Bloomberg\Futures"
helper_directory = r'G:\Projects\BackTesting1.0\Data\Bloomberg\HelperFiles'

bt_folder = r'BackTests\rsi_mr_jk'

# Define paths to auxiliary data for MarketData
tick_values_path = os.path.join(helper_directory, 'fut_val_pt.parquet')
fx_rates_path = os.path.join(helper_directory, 'fxHist.parquet')

# Initialize the MarketData
market_data = MarketData(
    base_directory=save_directory,
    tick_values_path=tick_values_path,
    fx_rates_path=fx_rates_path,
    instrument_type="Futures",
    n_threads=8,  # Number of threads for parallel data loading
    log_level=logging.ERROR  # Set to DEBUG for more detailed logs
)


In [2]:
tick = 'XTD1 Curncy'
# Access data for a specific ticker
try:
    test_df = market_data.get_ticker_data(tick)
    print(test_df)
except ValueError as e:
    print(e)

# Access all preprocessed data
all_data = market_data.get_data()
print(f"Total tickers loaded: {len(all_data)}")

# Access FX rates
fx_rates = market_data.get_fx_rates()
# Access tick values
tick_values = market_data.get_tick_values()
# Access asset classes
asset_classes = market_data.get_asset_classes()

#market_data = market_data.date_filter(start_date='01012010')

shape: (2_371, 14)
┌────────────┬────────┬────────┬────────┬───┬─────────┬─────────┬─────────────────┬────────────────┐
│ Date       ┆ Open   ┆ High   ┆ Low    ┆ … ┆ BadOHLC ┆ FX_Rate ┆ Tick_Value_Base ┆ Tick_Value_USD │
│ ---        ┆ ---    ┆ ---    ┆ ---    ┆   ┆ ---     ┆ ---     ┆ ---             ┆ ---            │
│ date       ┆ f64    ┆ f64    ┆ f64    ┆   ┆ bool    ┆ f64     ┆ f64             ┆ f64            │
╞════════════╪════════╪════════╪════════╪═══╪═════════╪═════════╪═════════════════╪════════════════╡
│ 2015-09-01 ┆ null   ┆ null   ┆ null   ┆ … ┆ false   ┆ 1.0     ┆ 100000.0        ┆ 100000.0       │
│ 2015-09-02 ┆ null   ┆ null   ┆ null   ┆ … ┆ false   ┆ 1.0     ┆ 100000.0        ┆ 100000.0       │
│ 2015-09-03 ┆ null   ┆ null   ┆ null   ┆ … ┆ false   ┆ 1.0     ┆ 100000.0        ┆ 100000.0       │
│ 2015-09-04 ┆ null   ┆ null   ┆ null   ┆ … ┆ false   ┆ 1.0     ┆ 100000.0        ┆ 100000.0       │
│ 2015-09-07 ┆ null   ┆ null   ┆ null   ┆ … ┆ false   ┆ 1.0     ┆ 100000

In [3]:

pSizeParams = {
    'AssetVol': 5_000_000,  # Target asset volatility in USD
    'VolLookBack': 21  # Lookback period for volatility calculation
}


In [4]:
from btEngine2.Rules.MeanReversion.ratioMR import *

fi_asst = 'WN1 Comdty'
# Define the list of asset pairs you want to trade
pairs = [
    ('NQ1 Index', fi_asst),
    (fi_asst, 'NQ1 Index')
]
    
strategy_params2 = {
    'pairs': pairs,
    'N': 5,
    'rsi_period': 3,
    'rsi_threshold': 10.0,
    'market_data': market_data,  # Pass the MarketData instance
}

   
strategy_params_short = {
    'pairs': pairs,
    'N': 5,
    'rsi_period': 3,
    'rsi_threshold': 10.0,
    'market_data': market_data,  # Pass the MarketData instance
}
# Set strategy parameters
strategy_params = {
    'pairs': pairs,
    'N': 5,
    'rsi_period': 3,
    'rsi_threshold': 10.0,
    'lmt_order': True,
    'lmt_day': 2,
    'lmt_day_only': False,
    'lmt_atr': 1,
    'lmt_epsilon': 0.1,
    'atr_period': 5,
    'atr_type': 'atr',
    'market_data': market_data  # Pass the MarketData instance
}

strategy_params3 = strategy_params2.copy()
strategy_params3['trend_filter'] = 120

strategy_params4 = strategy_params2.copy()
strategy_params4['oversold_cond'] = True

strategy_params5 = strategy_params3.copy()
strategy_params5['oversold_cond'] = True


In [5]:
### Where the magic happens

asst_main = 'NQ1 Index'

cmp_assts = market_data.get_asset_classes()['eq-us']
cmp_assts
cmp_assts = ['US1 Comdty', 'WN1 Comdty']


respnls = {}
results = {}
statsdf = {}
trules = {}

for asst in cmp_assts:
    
    pairs = [
        (asst_main, asst),
        (asst, asst_main)
    ]
    
    strategy_params2 = {
        'pairs': pairs,
        'N': (5, 2),
        'rsi_period': (3,3),
        'rsi_threshold': (10.0,8.0),
        'lmt_order': False,
        'lmt_day': 1,
        'lmt_day_only': True,
        'lmt_atr': 1.0,
        'lmt_epsilon': 0.1,
        'atr_period': 14,
        'atr_type': 'atr',
        'market_data': market_data,
        'risk_ratio': (1.0, 1.2)
    }

    tmp = TradingRule(
        market_data=market_data,
        trading_rule_function=ratioMR_rsi_long,
        trading_params=strategy_params2,
        position_sizing_params=pSizeParams,  # Define as needed
        incl_assets=[asst_main, asst],  # Include all involved assets
        name_label='TestMR_' + asst,
        strat_descr= f'rsi mr strategy {asst_main} vs. {asst}',
        log_level=logging.ERROR,
        bt_folder = bt_folder
    )

    res = tmp.backtest_all_assets(save=True)
    resdb = tmp.plot_equity(byassets=True, totalsys=True, start_date = '01012009', save_fig=False)
    stats = tmp.calculate_statistics(byassets=True, totalsys=True, start_date='01012009')
    statsdf[asst] = stats
    results[asst] = resdb
    respnls[asst] = res
    trules[asst] = tmp

# Extract the desired statistics from each dataframe in statsdf

required_columns = ['Average Ann. PnL', 'Average Ann. Vol', 'Sharpe Ratio (ann.)', 'Sortino Ratio (ann.)', 'Hit Rate (%)', 'Profit Factor', 
                    'Max drawdown', 'Average drawdown', 'Avg Drawdown Duration', 'Drawdowns per year', 'Max Drawdown Duration']
extracted_stats = {key: statsdf[key].loc['Total', [col for col in required_columns if col in statsdf[key].columns]] for key in statsdf.keys()}

# Convert the dictionary to a dataframe
extracted_stats_df = pd.DataFrame.from_dict(extracted_stats, orient='index')
extracted_stats_df.to_clipboard()
extracted_stats_df

,Average Ann. PnL,Average Ann. Vol,Sharpe Ratio (ann.),Sortino Ratio (ann.),Hit Rate (%),Profit Factor,Max drawdown,Average drawdown,Avg Drawdown Duration,Drawdowns per year
US1 Comdty,4.289488e+06,3.285225e+06,1.305691,0.961227,54.226475,1.483510,4057320.0,1.044663e+06,23.206349,7.971880
WN1 Comdty,4.605648e+06,3.301399e+06,1.395059,1.042753,63.521127,1.789599,4322165.0,1.064005e+06,24.174312,7.385856


In [6]:
respnls['US1 Comdty'][0]['US1 Comdty'].to_clipboard()

In [7]:
nq_pl = market_data.get_ticker_data('WN1 Comdty')
nq = nq_pl.to_pandas() 
nq.set_index('Date', inplace=True)

In [8]:
import plotly.graph_objects as go

# Extract the OHLC data for 'NQ1 Index'
nq_data = nq[['Open', 'High', 'Low', 'Close']]

# Create the OHLC plot
fig = go.Figure(data=[go.Ohlc(
    x=nq_data.index,
    open=nq_data['Open'],
    high=nq_data['High'],
    low=nq_data['Low'],
    close=nq_data['Close'],
    name='NQ1 Index'
)])

# Update layout for better visualization
fig.update_layout(
    title='NQ1 Index OHLC Chart',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False
)

# Show the plot
fig.show()

In [9]:
mthlypnl = trules['WN1 Comdty'].monthly_pnl(filter_assets=['WN1 Comdty'],save=True)

In [10]:
mthlypnl

,1,2,3,4,5,6,7,8,9,10,11,12,Annual_PnL,Sharpe_Ratio,Annual_Vol,Hit_Rate,Profit_Factor,Worst_Drawdown
Year,,,,,,,,,,,,,,,,,,
2010,0.0,-1106362.5,-27787.5,446362.5,-359550.0,228375.0,393375.0,0.0,573825.0,-980550.0,267225.0,632737.5,67650.0,0.040172,1.732136e+06,0.520000,1.020094,-1256475.0
2011,432675.0,293137.5,-100912.5,-113475.0,559200.0,34650.0,1055475.0,64350.0,134137.5,473437.5,0.0,-592687.5,2239987.5,1.335116,1.677748e+06,0.629630,1.957812,-592687.5
2012,170250.0,202200.0,-142312.5,581475.0,0.0,144562.5,849787.5,155850.0,-1311075.0,-733725.0,-171337.5,-802575.0,-1056900.0,-0.499888,2.105917e+06,0.419355,0.790234,-3018712.5
2013,242925.0,0.0,-730200.0,516937.5,-43425.0,-151087.5,707550.0,0.0,-136425.0,14212.5,628125.0,103612.5,1152225.0,0.780616,1.476045e+06,0.680000,1.493947,-747000.0
2014,0.0,37762.5,-231675.0,-820575.0,-364837.5,-114525.0,977250.0,704625.0,531675.0,-603825.0,473325.0,124987.5,714187.5,0.394760,1.809171e+06,0.555556,1.197458,-2062650.0
2015,112125.0,-1268775.0,226875.0,137287.5,329475.0,772462.5,265725.0,-75525.0,448050.0,53325.0,225000.0,601462.5,1827487.5,1.277749,1.424586e+06,0.653846,1.888688,-1399875.0
2016,0.0,1592325.0,306900.0,397837.5,348750.0,1592400.0,206775.0,418125.0,0.0,-321375.0,-610350.0,-506812.5,3424575.0,1.421688,2.408809e+06,0.619048,2.327799,-1461262.5
2017,824550.0,163125.0,73125.0,812250.0,168637.5,0.0,150562.5,699750.0,1446825.0,801300.0,-717337.5,-107475.0,4315312.5,1.941840,2.231134e+06,0.702703,2.347451,-992287.5
2018,-1170600.0,464550.0,-132675.0,169725.0,-162487.5,-244950.0,629775.0,921787.5,143550.0,226687.5,-492112.5,844875.0,1198125.0,0.489746,2.446423e+06,0.545455,1.233322,-1279500.0


In [11]:
statsdf = trules['WN1 Comdty'].calculate_statistics(byassets=True, totalsys=True, start_date='01012000', save=True)